In [1]:
# from dataprocess.kdd_cup99 import KDD_CUP_99_DataLoader
# from dataprocess.unsw_nb15 import UNSW_NB15_DataLoader
from dataprocess.cic_ids_2017 import CIC_IDS_2107_DataLoader
from net.CNN import CNN3_4
# from net.FullConnect import linear3_dropout
# from net.multCNN import multCNN

In [2]:
import torch
from torch import squeeze
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [3]:
import os 
import time

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0'

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

t = time.localtime()
year, month, day = t.tm_year, t.tm_mon, t.tm_mday

net = input('input the name of net to train: ')
mode = input('input the tag about the training: ')
if mode == '':
    mode = 'no_tag'

log_directory = f"/Transfer-learning-IDS/log/exp/{net}/{mode}/{month}_{day}/"
ckpt_directory = f"/Transfer-learning-IDS/history/{net}/{mode}/checkpoint/{month}_{day}/"
if not os.path.isdir(log_directory):
    os.makedirs(log_directory)
if not os.path.isdir(ckpt_directory):
    os.makedirs(ckpt_directory)
print('log_directory: ', log_directory)
print('ckpt_directory: ', ckpt_directory)

cuda
log_directory:  /Transfer-learning-IDS/log/exp/CNN3_4/CIC_IDS2017_UNSW_NB15_transfer_Adam_rebalance_no_frozen/5_12/
ckpt_directory:  /Transfer-learning-IDS/history/CNN3_4/CIC_IDS2017_UNSW_NB15_transfer_Adam_rebalance_no_frozen/checkpoint/5_12/


In [4]:
# # for kdd_cup99
# dataset = KDD_CUP_99_DataLoader('E:/DataSets/kddcup.data', 256)
# data_labels = dataset.data.to_num_column_dic[41].keys()
# class_num = len(data_labels)
# discrete_column_idx = dataset.data.discrete_column
# discrete_column_idx.remove(41)
# continuous_column_idx = [i for i in range(41) if i not in discrete_column_idx]
# input_channel_kind = len(discrete_column_idx)
# input_channel_num = len(continuous_column_idx)

# print(discrete_column_idx, continuous_column_idx, data_labels)

In [5]:
# # for UNSW_NB15
# dataset = UNSW_NB15_DataLoader('E:/DataSets/UNSW-NB15 - CSV Files', 256, mode='Train', rebalanced=True)
# data_labels = dataset.data.label_category.keys()
# class_num = len(data_labels)
# # discrete_column_idx = dataset.data.discrete_column
# # discrete_column_idx.remove(47)
# # discrete_column_idx.remove(48)
# # continuous_column_idx = [i for i in range(47) if i not in discrete_column_idx]
# # input_channel_kind = len(discrete_column_idx)
# # input_channel_num = len(continuous_column_idx)

# # hight, width = dataset.data.image_hight, dataset.data.image_width
# feature_length = dataset.data.feature_length
# print(class_num, feature_length)

In [6]:
# for cic_ids2017
batch_size = 256
dataset = CIC_IDS_2107_DataLoader('E:/DataSets/CIC-IDS2016', batch_size, mode='Train')
data_labels = dataset.data.label_category.keys()
class_num = len(data_labels)
feature_length = dataset.data.feature_length
print(class_num, feature_length)

9 85


In [7]:
writer1 = SummaryWriter(log_directory)

In [9]:
net = CNN3_4(feature_length, class_num).to(device)
epoch = 100
criterion = nn.CrossEntropyLoss().to(device)
# optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

In [10]:
# net = linear3_Relu(features_num, class_num).to(device)
# net = CNN(input_channel_num + input_channel_kind, class_num).to(device)
net = CNN3_4(feature_length, class_num).to(device)
# net = linear3_dropout(feature_length, class_num).to(device)

ckpt = torch.load('history/CNN3_4/UNSW_NB15_Adam_rebalance/checkpoint/5_12/ckpt_best_49.pth')

part_params = net.state_dict()
for name, param in ckpt['net'].items():
    if 'cnn' in name:
    # if 'fc_1' in name:
        part_params[name] = param

# for name, param in net.named_parameters():
#     if "cnn" in name:
#     # if 'fc_1' in name:
#         param.requires_grad = False

# # epoch_start = ckpt["epoch"]
net.load_state_dict(part_params)
# optimizer.load_state_dict(ckpt["optimizer"])

epoch = 50
criterion = nn.CrossEntropyLoss().to(device)
# optimizer = optim.SGD(filter(lambda p : p.requires_grad, net.parameters()), lr=0.01)
# optimizer = optim.SGD(net.parameters(), lr=0.001)
# optimizer = optim.Adam(filter(lambda p : p.requires_grad, net.parameters()), lr=0.001)
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

In [11]:
for t in range(0, epoch):
    running_loss = 0
    for step, (x, y) in enumerate(dataset):
        
        x = x.to(device)
        y = y.to(device)
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()

        y_pred = net(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        
        # visualize loss
        running_loss += loss.item()
    # ...log the running loss
    writer1.add_scalar('training loss', running_loss, t)
    scheduler.step()
    if t % 10 == 9:
        checkpoint = {"net": net.state_dict(), 'optimizer':optimizer.state_dict(), "epoch": t}
        torch.save(checkpoint,  ckpt_directory + 'ckpt_best_%s.pth' %(str(t)))
writer1.close()

epoch: 0, loss: 777.298563, learning rate: 0.001000
epoch: 1, loss: 750.731334, learning rate: 0.001000
epoch: 2, loss: 748.579432, learning rate: 0.001000
epoch: 3, loss: 748.095240, learning rate: 0.001000
epoch: 4, loss: 747.177420, learning rate: 0.001000
epoch: 5, loss: 747.052834, learning rate: 0.001000
epoch: 6, loss: 747.336188, learning rate: 0.001000
epoch: 7, loss: 746.871989, learning rate: 0.001000
epoch: 8, loss: 746.774409, learning rate: 0.001000
epoch: 9, loss: 746.796722, learning rate: 0.001000
epoch: 10, loss: 746.701061, learning rate: 0.001000
epoch: 11, loss: 746.696026, learning rate: 0.001000
epoch: 12, loss: 748.697738, learning rate: 0.001000
epoch: 13, loss: 746.933248, learning rate: 0.001000
epoch: 14, loss: 746.580580, learning rate: 0.001000
epoch: 15, loss: 735.225235, learning rate: 0.001000
epoch: 16, loss: 716.196510, learning rate: 0.001000
epoch: 17, loss: 716.147559, learning rate: 0.001000
epoch: 18, loss: 715.975003, learning rate: 0.001000
epo

In [ ]:
# epoch 20
# epoch 40
# epoch 18